# Notebook de pré-processamento da planilha IW47

### Importando bibliotecas necessárias

In [ ]:
from resolve_path import ajuste_path, read_input
from unidecode import unidecode

### Leitura da planilha

In [ ]:
path = "data/"
pathUtil = path + "util/"
pathUtil = ajuste_path(pathUtil)

df = read_input("IW47-Confirmações_Executadas_2020-a-28jun24.csv")

### Pré-processamento da coluna 'Txt.breve operação'

In [ ]:
print(df["Txt.breve operação"].value_counts())

In [ ]:
# Converte coluna 'Txt.breve operação' para lowercase e remove acentos
df["Txt.breve operação"] = df["Txt.breve operação"].apply(
    lambda x: unidecode(x).lower() if isinstance(x, str) else x)


# Refatora coluna 'Txt.breve operação'
# Por exemplo:
#    Txt.breve operação    Output
# 1: "servico-6a"       => "servico"
df["Txt.breve operação"] = df["Txt.breve operação"].str.split("-").str[0]
df["Txt.breve operação"] = df["Txt.breve operação"].str.split(" ").str[0]

#### Transforma linhas com o prefixo das operações mais comuns para o seu nome padrão

In [ ]:
dict_prefixos = {
    "serv": "servico",
    "analise": "servico",
    "analises": "servico",
    "analise/aprovacao": "servico",
    "chaparia": "servico",
    "execucao": "servico",
    "trabalho": "servico",
    "desl": "deslocamento",
    "prep": "preparo",
    "rel": "relatorio",
    "super": "supervisao",
    "lab": "laboratorio",
    "eng": "engenharia",
    "comiss": "comissionamento",
    "insp": "inspecao",
    "verif": "verificacao",
    "planej": "planejamento",
    "calib": "calibracao",
    "anormalidades": "anormalidade",
    "exec": "execucao",
}

# Atualiza os valores da coluna 'Txt.breve operação' de acordo com o dicionário 'dict_prefixos'


def padronizar_coluna(linha, dict_prefixos):
    for prefixo, palavra in dict_prefixos.items():
        if str(linha).startswith(prefixo):
            return palavra

    return linha


df["Txt.breve operação"] = df["Txt.breve operação"].apply(
    lambda x: padronizar_coluna(x, dict_prefixos))

#### Preprocessa nomes das colunas 
Usa-se o padrão de letras minusculas e sem espaços

In [ ]:
df.columns = [unidecode(coluna).lower().replace(" ", "_")
              for coluna in df.columns]

df.columns

### Salva o DataFrame da IW47 processado como csv

In [ ]:
# Salva o dataframe em um arquivo .csv
df.to_csv(pathUtil + "os/operacoes_preprocessada.csv",
          index=False, sep="@", encoding="utf-8")